In [5]:
import requests
import re
from bs4 import BeautifulSoup
import os

input_keyword = "網路爬蟲"  # 這裡可以自己定義有興趣的關鍵字

utf8_url = repr(input_keyword.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
print("%s: %s" % (input_keyword, utf8_url[2:-1:1]))     # 擷取中間的編碼結果

# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
print(root_keyword_link)

網路爬蟲: %E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2
/wiki/%E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2


In [10]:
def WikiArticle(key_word_link, key_word, recursive):
    
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word_link, key_word))
        
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': '/wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    

        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
        
        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #
       
        
        path='D:/WikiArticle/'
        if not os.path.isdir(path):

            os.mkdir(path)
        article=[]
        for paragraph in content:
            article.append(paragraph.get_text())
        with open(path+ key_word+'.txt', 'w',encoding="utf-8") as f:
            for item in article:
                f.write("%s\n" % item)
        
        

        
        #
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        external_link_dict = dict({})
        for ext_link in content:
            a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
            if len(a_tag) > 0:
                for link_string in a_tag:
                    a_link = link_string["href"]       # 外部連結的網址
                    a_keyword = link_string.get_text()  # 外部連結的中文名稱
                    external_link_dict[a_keyword]=a_link

                    
        #
        # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            
            recursive = recursive + 1  # 遞迴深度加1
            
            for k, v in external_link_dict.items():
                WikiArticle(v,k,recursive)  # 再次呼叫同樣的函數，執行同樣的流程
                

In [11]:
max_recursive_depth = 2

WikiArticle(root_keyword_link, input_keyword, 0)

遞迴層[0] - /wiki/%E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2 (網路爬蟲)
遞迴層[1] - /wiki/%E4%B8%87%E7%BB%B4%E7%BD%91 (全球資訊網)
遞迴層[2] - /wiki/%E5%9B%A0%E7%89%B9%E7%BD%91 (網際網路)
遞迴層[2] - /wiki/%E8%B6%85%E6%96%87%E6%9C%AC (超文字)
遞迴層[2] - /wiki/%E8%92%82%E5%A7%86%C2%B7%E4%BC%AF%E7%BA%B3%E6%96%AF-%E6%9D%8E (提姆·柏內茲-李)
遞迴層[2] - /wiki/CERN (CERN)
遞迴層[2] - /wiki/%E7%B6%B2%E9%A0%81%E7%80%8F%E8%A6%BD%E5%99%A8 (網頁瀏覽器)
遞迴層[2] - /wiki/%E8%B3%87%E8%A8%8A%E6%99%82%E4%BB%A3 (資訊時代)
遞迴層[2] - /wiki/%E7%BD%91%E9%A1%B5 (網頁)
遞迴層[2] - /wiki/%E6%A0%BC%E5%BC%8F%E5%8C%96%E6%96%87%E6%9C%AC (格式化)
遞迴層[2] - /wiki/%E8%B6%85%E6%96%87%E4%BB%B6%E6%A8%99%E7%A4%BA%E8%AA%9E%E8%A8%80 (超文件標示語言)
遞迴層[2] - /wiki/%E5%9C%96%E7%89%87 (圖片)
遞迴層[2] - /wiki/%E5%BD%B1%E7%89%87 (影片)
遞迴層[2] - /wiki/%E8%81%B2%E9%9F%B3 (聲音)
遞迴層[2] - /wiki/%E5%A4%9A%E5%AA%92%E9%AB%94 (多媒體)
遞迴層[2] - /wiki/ENQUIRE (ENQUIRE)
遞迴層[2] - /wiki/%E6%95%B0%E6%8D%AE%E5%BA%93 (資料庫)
遞迴層[2] - /wiki/%E8%AF%AD%E4%B9%89%E7%BD%91 (語意網)
遞迴層[2] - /wiki/%E7%BD%97%E4%BC%AF%E7%89%B9%C2%B7%E5%8D%A

FileNotFoundError: [Errno 2] No such file or directory: 'D:/WikiArticle/TCP/IP.txt'